In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it

from scipy.sparse import csr_matrix, coo_matrix


from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

import optuna


from utils import prefilter_items
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Задание 1

Прочитать статьи про BPR, WARP loss

+

Задание 2

Сделать грид серч текущей модели

In [3]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float)

sparse_user_item = csr_matrix(user_item_matrix)

In [5]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float)

test_sparse_user_item = csr_matrix(test_user_item_matrix)

In [6]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [7]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
user_feat.shape

(2499, 7)

In [9]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [10]:
item_feat.shape

(86865, 6)

In [11]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [12]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
scores = []

In [14]:
def objective(trial):
    global scores

    
    loss = 'warp'
    components = trial.suggest_int('components', 16, 64)
    lr = trial.suggest_uniform('lr', 0.0001, 0.5)
    item_alpha = 0.1
    user_alpha = 0.1
    model = LightFM(no_components=components,
                    loss=loss,
                    learning_rate=lr, 
                    item_alpha=item_alpha,
                    user_alpha=user_alpha, 
                    random_state=42)

    
    model.fit((sparse_user_item > 0) * 1,
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values),
              item_features=csr_matrix(item_feat_lightfm.values),
              epochs=15, 
              num_threads=6,
              verbose=False)

    
    train_precision = precision_at_k(model, sparse_user_item, 
                                     user_features=csr_matrix(user_feat_lightfm.values),
                                     item_features=csr_matrix(item_feat_lightfm.values),
                                     k=5).mean()
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix), 
                                     user_features=csr_matrix(user_feat_lightfm.values),
                                     item_features=csr_matrix(item_feat_lightfm.values),
                                     k=5).mean()

    
    scores.append({'components': components,
                   'lr': lr,
                   'train': train_precision,
                   'test': test_precision})

    return test_precision


study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=25) 

[I 2022-05-01 19:17:47,536] A new study created in memory with name: no-name-30c49515-32a3-4837-a353-3f0b4ce1b2d9
[I 2022-05-01 19:27:02,505] Trial 0 finished with value: 0.004703576676547527 and parameters: {'components': 29, 'lr': 0.281429826138992}. Best is trial 0 with value: 0.004703576676547527.
[I 2022-05-01 19:42:30,726] Trial 1 finished with value: 0.005879471078515053 and parameters: {'components': 49, 'lr': 0.1703878984266359}. Best is trial 1 with value: 0.005879471078515053.
[I 2022-05-01 19:50:03,446] Trial 2 finished with value: 0.004115629475563765 and parameters: {'components': 20, 'lr': 0.2963899187745479}. Best is trial 1 with value: 0.005879471078515053.
[I 2022-05-01 20:02:03,235] Trial 3 finished with value: 0.0006859382847324014 and parameters: {'components': 51, 'lr': 0.3636862103813766}. Best is trial 1 with value: 0.005879471078515053.
[I 2022-05-01 20:13:27,489] Trial 4 finished with value: 0.0027437531389296055 and parameters: {'components': 48, 'lr': 0.3672

Оптимальными параметрами модели являются: no_components = 60, learning_rate = 0.149